In [ ]:
# Hydrofabric extent and soil attributes investigation for running in CFE (all flavors) or the LSTM
Jeremy Rapp, 6/27/2023, rappjer1@gmail.com

## Section 1: Getting appropriate CAMELS shapefile data and basin lists to extract the correct basins

In [ ]:
#Loc of shape files
CAMELSshpLoc = r'H:/Shared drives/SI_NextGen_Aridity/hydrofabric/basin_set_full_res/'
CAMELSshpFile = 'HCDN_nhru_final_671.shp'
basinSelect= 'basin_list_516.txt'



#Read in shape file look at attributes
CAMELSshp = gpd.read_file(CAMELSshpLoc+CAMELSshpFile)
CAMELSshp['gauge_id_len'] = CAMELSshp['hru_id'].map(str).apply(len)
#print(allSemiArid['gauge_id_len'].unique())
CAMELSshp['gauge_id_str'] = CAMELSshp['hru_id'].map(str)
CAMELSshp.loc[CAMELSshp['gauge_id_len'] != 8, 'gauge_id_str'] = '0' + CAMELSshp['gauge_id_str']
print(CAMELSshp.head())
print(CAMELSshp.describe())

#Read in basin list
basinList = pd.read_csv(CAMELSshpLoc+basinSelect, header=None, names=['gauge_id'], dtype=str)
basinList['gauge_id_len'] = basinList['gauge_id'].map(str).apply(len)
#print(allSemiArid['gauge_id_len'].unique())
basinList['gauge_id_str'] = basinList['gauge_id'].map(str)
basinList.loc[basinList['gauge_id_len'] != 8, 'gauge_id_str'] = '0' + basinList['gauge_id_str']
basinList.head()

# Now mask the shape file to only include the basins in the basin list, largely because masking is typically faster
mask = CAMELSshp['gauge_id_str'].isin(basinList['gauge_id'])
filtered_geodf = CAMELSshp[mask]
filtered_geodf['gauge_id'] = 'Gage-' + filtered_geodf['gauge_id_str'].astype(str)
print(filtered_geodf.head())


# Now build a dictionary of the locations for the catchment attributes from hydrofabric
hydrofabrics_loc = "G:\\Shared drives\\SI_NextGen_Aridity\\data\\CAMELS_Hydrofabric_CUAHSIsubset\\Hydrofabrics\\Hydrofabrics\\"

def search_files_by_name(root_dir, target_file_name, identifying_substring):
    # Dictionary to store the paths of the files with target_file_name
    # Key: Identifying_Directory, Value: full file path
    file_paths = {}

    # Walk through the root directory
    for current_dir, _, files in os.walk(root_dir):
        for file in files:
            # Check if the file name matches the target file name
            if file == target_file_name:
                # Full path of the file
                full_file_path = os.path.join(current_dir, file)
                
                # Extract the Identifying_Directory part of the file path
                path_parts = os.path.normpath(full_file_path).split(os.sep)
                identifying_directory = None
                
                # Iterate through path parts to find the identifying_directory
                for part in path_parts:
                    if part.startswith(identifying_substring):
                        identifying_directory = part
                        break

                # Add the full file path to the dictionary
                if identifying_directory:
                    file_paths[identifying_directory] = full_file_path

    return file_paths

# Example usage
root_directory = hydrofabrics_loc   # Replace this with the path to your root directory
target_file_name = 'catchments.geojson'              # Replace this with the specific file name you are looking for
identifying_substring = 'Gage'                   # The substring the identifying directory starts with

result = search_files_by_name(root_directory, target_file_name, identifying_substring)

In [ ]:
#Add a column for hydrofabrics area from the individual config geojson files
# Each total catchment in the hydrofabric will need to be summed from its smaller parts

filtered_geodf['area_CAMELSsqkm'] = filtered_geodf['AREA']/1000000
filtered_geodf['area_hydrofabric'] = None

for id, filepath in result.items():
    gdf = gpd.read_file(filepath)
    sum_areasqkm = gdf['areasqkm'].sum()
    #print(id)
    filtered_geodf.loc[filtered_geodf['gauge_id'] == id, 'area_hydrofabric'] = sum_areasqkm

print(filtered_geodf.head())


## Section 2: Plotting the CAMELS catchments vs the Hydrofabric lumped basins to see how they compare

In [ ]:
matplotlib.rcParams.update(_VSCode_defaultMatplotlib_Params)
plt.style.context('seaborn-v0_8-talk')
g = sns.FacetGrid(filtered_geodf, height=6)
g = g.map(plt.scatter, "area_CAMELSsqkm", "area_hydrofabric", edgecolor="w")
max_value = filtered_geodf['area_CAMELSsqkm'].max()
g = g.map(lambda *args, **kwargs: plt.plot([0, max_value], [0, max_value], 'r--'))
g.set(title='CAMELS Catchment Area to Lumped Hydrofabric Area (sqKm) \n Updated 6/30/2023', xlabel='CAMELS Area (sqKm)', ylabel='Hydrofabric Area (sqKm)')

# Count the number of points
num_points = len(filtered_geodf)

# Add the number of points as text to the plot
# x_position and y_position should be chosen based on the range of your data
x_position = 0.04 * max_value
y_position = 15 * max_value
plt.text(x_position, y_position, f'Total Catchments from original Export : {num_points}', fontsize=12)
plt.show()

In [ ]:
filtered_geodf['area_similarity'] = filtered_geodf['area_hydrofabric'] / filtered_geodf['area_CAMELSsqkm']
ratio_cut_off = 2.0
no_extremes = filtered_geodf[(filtered_geodf['area_similarity'] < ratio_cut_off) & (filtered_geodf['area_similarity'].notna())]
g = sns.FacetGrid(no_extremes, height=6)
g = g.map(plt.scatter, "area_CAMELSsqkm", "area_hydrofabric", edgecolor="w", label= 'Catchment Points')
max_value = filtered_geodf['area_CAMELSsqkm'].max()
g = g.map(lambda *args, **kwargs: plt.plot([0, max_value], [0, max_value], 'r--', label='1 to 1 Line'))
g.set(title='Lumped Hydrofabric Area (km\u00b2) vs CAMELS Catchment Area (km\u00b2) \n Updated 7/3/2023', xlabel='CAMELS Area (km\u00b2)', ylabel='Hydrofabric Area (km\u00b2)')


# title
plt.legend(title='Legend')

new_title = 'Legend'
# legend = g._legend
# legend.set_title(new_title)

# # replace labels
# new_labels = ['Catchment Points', '1 to 1 Line']
# for t, l in zip(legend.texts, new_labels):
#     t.set_text(l)

# Count the number of points
num_points = len(no_extremes)

# Add the number of points as text to the plot
# x_position and y_position should be chosen based on the range of your data
x_position = 0.4 * max_value
y_position = 0.04 * max_value
plt.text(x_position, y_position, f'Total Catchments (excluding Hydrofabric lumped\n catchments {ratio_cut_off}x larger than CAMELS)  : {num_points}', fontsize=10)
plt.show()